<a href="https://colab.research.google.com/github/sakethsridhara/Milwaukee_bearings/blob/main/latentSpaceBearingsPlotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bearing Selection through latent space

In [73]:
# run this first time to clone the directory
!git clone https://github.com/sakethsridhara/Milwaukee_bearings.git

Cloning into 'Milwaukee_bearings'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 70 (delta 28), reused 53 (delta 16), pack-reused 0
Receiving objects: 100% (70/70), 2.37 MiB | 8.40 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [74]:
from os import chdir # to alter your working directory
chdir('/content/Milwaukee_bearings') # setting the working directory


In [177]:
import numpy as np
import pandas as pd
import torch
import sys
import time
import matplotlib.pyplot as plt
import matplotlib
from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon, Ellipse
import plotly.graph_objects as go
sys.path.insert(0,'/content/Milwaukee_bearings/src/')
from utilFuncs import to_np, to_torch
from materialEncoder import MaterialEncoder
from smallestEllipse import *
from plotly import __version__
import seaborn as sns
import scipy as sp
import copy


### Bearing database

In [76]:
def preprocessData():
  df = pd.read_excel('./data/bearingData.xlsx')
  dataIdentifier = {'classID':df[df.columns[0]],'name': df[df.columns[1]]} # name of the material and type
  trainInfo = np.log10(df[df.columns[2:]].to_numpy())
  dataScaleMax = torch.tensor(np.max(trainInfo, axis = 0))
  dataScaleMin = torch.tensor(np.min(trainInfo, axis = 0))
  normalizedData = (torch.tensor(trainInfo) - dataScaleMin)/(dataScaleMax - dataScaleMin)
  trainingData = normalizedData.clone().float()
  dataInfo = {'ID':{'idx':0,'scaleMin':dataScaleMin[0], 'scaleMax':dataScaleMax[0]},\
              'OD':{'idx':1,'scaleMin':dataScaleMin[1], 'scaleMax':dataScaleMax[1]},\
              'Width':{'idx':2,'scaleMin':dataScaleMin[2], 'scaleMax':dataScaleMax[2]},\
              'DynamicLoadRating':{'idx':3,'scaleMin':dataScaleMin[3], 'scaleMax':dataScaleMax[3]},\
              'StaticLoadRating':{'idx':4,'scaleMin':dataScaleMin[4], 'scaleMax':dataScaleMax[4]},\
              'RPM':{'idx':5,'scaleMin':dataScaleMin[5], 'scaleMax':dataScaleMax[5]},\
              'Cost':{'idx':6,'scaleMin':dataScaleMin[6], 'scaleMax':dataScaleMax[6]}}
  return trainingData, dataInfo, dataIdentifier, trainInfo
trainingData, dataInfo, dataIdentifier, trainInfo = preprocessData()
numMaterialsInTrainingData, numFeatures = trainingData.shape

In [164]:
latentDim = 2
hiddenDim = [250,250]
numEpochs = 30000
klFactor = 5e-5
learningRate = 2e-3
vaeSettings = {'encoder':{'inputDim':numFeatures, 'hiddenDim':hiddenDim,\
                                          'latentDim':latentDim},\
               'decoder':{'latentDim':latentDim, 'hiddenDim':hiddenDim,\
                                          'outputDim':numFeatures}}
materialEncoder = MaterialEncoder(trainingData, dataInfo, dataIdentifier, vaeSettings)
savedNet = './data/vaeTrained.pt'
materialEncoder.vaeNet = torch.load(savedNet,map_location=torch.device('cpu'))

# start = time.perf_counter()
# convgHistory = materialEncoder.trainAutoencoder(numEpochs, klFactor, savedNet, learningRate)
# print('training time : {:.2F} '.format(time.perf_counter() - start))

In [ ]:
# @title
def plotConvergence(convg):
  plt.figure();
  strokes = ['--', '-.', '-', ':']
  for ctr, key in enumerate(convg):
    y = torch.as_tensor(convg[key]).detach().numpy()
    y_mvavg = np.convolve(y, np.ones(20), 'valid') / 20.
    plt.semilogy(y_mvavg, strokes[ctr], label = str(key))
    plt.xlabel('Iterations')
    plt.ylabel(str(key))
    plt.grid('True')
    plt.legend()



  # plotly_fig = tls.mpl_to_plotly(f1) ## convert
  # iplot(plotly_fig)
plotConvergence(convgHistory)

In [ ]:
# @title
def plotConvergence2(convg):
    fig = go.Figure()
    strokes = ['dash', 'dashdot', 'solid', 'dot']  # Corresponding to ['--', '-.', '-', ':']

    for ctr, key in enumerate(convg):
        y = torch.as_tensor(convg[key]).detach().numpy()
        y_mvavg = np.convolve(y, np.ones(20), 'valid') / 20.

        fig.add_trace(go.Scatter(
            x=list(range(len(y_mvavg))),
            y=y_mvavg,
            mode='lines',
            line=dict(dash=strokes[ctr], width=2),
            name=str(key)
        ))

    fig.update_layout(
        xaxis_title='Iterations',
        yaxis_title=str(key),
        showlegend=True,
        legend=dict(orientation="h", x=0.1, y=1.1),
        yaxis_type="log")

    fig.show(renderer='colab')

# Example usage
# Assuming 'convg' is a dictionary with keys and corresponding lists of convergence data
plotConvergence2(convgHistory)


In [78]:
matidxs = np.arange(trainInfo.shape[0]).astype(int)
props = ['ID','OD','Width','StaticLoadRating','DynamicLoadRating','RPM','Cost']

def unnormalize(val, minval ,maxval):
  return 10.**(minval + (maxval-minval)*val)
def decodeAll():
  vae = materialEncoder.vaeNet
  decoded = vae.decoder(vae.encoder.z)
  matProp = {'ID':None,'OD':None,'Width':None,'StaticLoadRating':None,'DynamicLoadRating': None, 'RPM':None,'Cost':None}

  for k in props:
    idx = materialEncoder.dataInfo[k]['idx']
    scaleMax = materialEncoder.dataInfo[k]['scaleMax']
    scaleMin = materialEncoder.dataInfo[k]['scaleMin']
    matProp[k] = unnormalize(decoded[:,idx], scaleMin ,scaleMax)#scaleMin + decoded[:,idx]*(scaleMax - scaleMin)
  return matProp

matProp = decodeAll()
# print('\n \n \t \t ------RECONSTRUCTED DATA----------')
# print('Catalog Name', end = '\t')
# for p in props:
#     print(p, end = '\t')

# for i in matidxs:
#   print(f"\n {dataIdentifier['name'][i]} \t ", end = '')
#   for p in props:
#     print('\t {:.2E}'.format(matProp[p][i]), end='')

merr = -1000000000.

maxError = {'ID':merr,'OD':merr,'Width':merr,'StaticLoadRating':merr,'DynamicLoadRating':merr, 'RPM':merr,'Cost':merr}
print('\n \n \t \t ------RECON ERROR (%)----------')
print('Catalog name', end = '\t')
errList = torch.zeros(trainInfo.shape[0],7);
for p in props:
    print(f"\t{p}", end = '\t')
for i in range(trainInfo.shape[0]):
  count = 0;

  if(i in matidxs): #
    print(f"\n  {dataIdentifier['name'][i]} ", end = '')

  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    trueData = 10**trainInfo[i,idx]
    reconData = matProp[p][i]
    err = torch.abs(100.*(trueData - reconData)/trueData).to('cpu')
    errList[i,count] = err
    count = count + 1;
    if(err > maxError[p]):
      maxError[p] = err
    if(i in matidxs):
      print('\t {:.1F} \t'.format(err), end='')


print('\n max Error', end = '')
for p in props:
  print('\t {:.1F}'.format(maxError[p]), end='')

print("\n Mean Error:")
print(torch.mean(errList,0))


 
 	 	 ------RECON ERROR (%)----------
Catalog name		ID		OD		Width		StaticLoadRating		DynamicLoadRating		RPM		Cost	
  60355K124 	 0.2 		 0.3 		 0.2 		 0.9 		 0.7 		 0.4 		 0.6 	
  60355K132 	 0.6 		 0.0 		 1.1 		 0.6 		 2.5 		 0.7 		 0.4 	
  60355K151 	 1.1 		 2.3 		 0.1 		 0.4 		 2.1 		 1.1 		 1.4 	
  60355K165 	 0.5 		 0.3 		 0.3 		 0.1 		 1.5 		 1.6 		 0.7 	
  60355K173 	 2.0 		 1.4 		 0.2 		 2.0 		 3.3 		 0.7 		 0.6 	
  60355K178 	 0.2 		 0.2 		 1.8 		 0.2 		 0.6 		 1.7 		 0.7 	
  60355K185 	 0.1 		 1.0 		 0.7 		 1.7 		 2.5 		 0.3 		 0.1 	
  60355K508 	 0.7 		 0.1 		 0.8 		 0.4 		 0.8 		 2.4 		 0.4 	
  60355K509 	 2.1 		 0.2 		 0.7 		 0.4 		 0.2 		 0.3 		 0.8 	
  60355K211 	 0.0 		 0.1 		 1.1 		 1.3 		 1.8 		 1.3 		 1.6 	
  60355K22 	 1.0 		 1.3 		 0.7 		 1.9 		 2.2 		 0.8 		 1.6 	
  60355K511 	 0.6 		 0.8 		 0.3 		 0.9 		 3.4 		 0.7 		 1.1 	
  60355K512 	 0.7 		 1.5 		 0.6 		 3.2 		 2.3 		 0.3 		 4.6 	
  5972K91 	 0.1 		 0.1 		 0.0 		 0.5 		 2.0 		 0.4 		 1.1 	
  5972K93 	 2.6 		

In [162]:
# @title
def plotlyLatent():
    clrs = ['blue', 'green']
    mrkrSet = [0,2]
    bearingClass = ['Ball Bearing','Roller Bearing']
    colorcol = dataIdentifier['classID']
    ptLabel = dataIdentifier['name']
    autoencoder = materialEncoder.vaeNet
    z = autoencoder.encoder.z.to('cpu').detach().numpy()
    scatter_traces = []
    for i in range(np.max(colorcol)+1):
      zMat = np.vstack((z[colorcol == i,0], z[colorcol == i,1])).T
      txt = ptLabel[colorcol == i]
      scatter_trace = go.Scatter(
                                  x=zMat[:, 0],
                                  y=zMat[:, 1],
                                  mode='markers',
                                  text = txt,
                                  name = bearingClass[i],
                                  marker=dict(size=8,
                                              color=clrs[i],
                                              colorscale='Viridis',
                                              opacity=0.7,
                                              symbol = mrkrSet[i]))
      scatter_traces.append(scatter_trace)
      hull = ConvexHull(zMat)
      cent = np.mean(zMat, 0)
      pts = []
      for pt in zMat[hull.simplices]:
          pts.append(pt[0].tolist())
          pts.append(pt[1].tolist())

      pts.sort(key=lambda p: np.arctan2(p[1] - cent[1],
                                      p[0] - cent[0]))
      pts = pts[0::2]  # Deleting duplicates
      enclosing_ellipse = welzl(np.array(pts, dtype=float))
      # plot resulting ellipse
      center,a,b,t = enclosing_ellipse

      elli = sample_ellipse(enclosing_ellipse,1000)
      scatter_trace = go.Scatter(x=elli[:, 0],
          y=elli[:, 1],
          mode= 'lines',
          text = bearingClass[i],
          hoverinfo ='skip',
          showlegend= False,
          marker=dict(size=8,
              color=clrs[i],
              colorscale='Viridis',
              opacity=0.7))
      scatter_traces.append(scatter_trace)



    scatter_plotly = go.Figure(data=scatter_traces)
    scatter_plotly.update_layout(plot_bgcolor='white',showlegend=True)
    scatter_plotly.update_layout(title='Latent Space of Bearings',title_x=0.5)

    scatter_plotly.update_layout(xaxis_range=[-3, 3],  # Set the x-axis limits
        yaxis_range=[-3, 3], width = 800, height = 800);
    scatter_plotly.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    scatter_plotly.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    scatter_plotly.update_layout(xaxis=dict(zerolinecolor='lightgrey'), yaxis=dict(zerolinecolor='lightgrey'))
    scatter_plotly.show(renderer='colab')
    return scatter_plotly



In [179]:
scatter_plotly = plotlyLatent()

In [165]:
def plotlyLatentContours(props = ['RPM']):
  n = 80
  zmin, zmax = -3,3
  xi = np.linspace(zmin, zmax, n)
  yi = np.linspace(zmin, zmax, n)
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim'])).to('cpu')
  Z[:,0], Z[:,1] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))

  vae = materialEncoder.vaeNet.to('cpu')
  trainData_z_np = to_np(vae.encoder.z)
  decoded = vae.decoder(Z)



  #-------------------------------------------#

  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matPropVal = to_np(10.**(scaleMin + decoded[:,idx]*(scaleMax - scaleMin)))
    levs = np.logspace(np.log10(min(matPropVal))*0.5, np.log10(max(matPropVal)), 40)
    contour_trace = go.Contour(z= matPropVal.reshape((n,n)), x=xi, y=yi,
        colorscale='Viridis',
        showscale=True,  # Hide the color scale
        contours=dict(coloring='lines', showlabels=True)
    )


    return contour_trace


    # surf = ax.contour(X, Y, (matPropVal.reshape((n,n))), levels = levs, cmap='viridis_r', alpha = 0.6)

    # surf = ax.contour(X, Y, (to_np(matPropVal).reshape((n,n))), levels = cutOff, cmap='coolwarm', alpha = 0.3)

    # surf = ax.contourf(X, Y, (matPropVal.reshape((n,n))), levels = cutOff, alpha = 0.2,\
    # colors=['g', 'g', '#C0C0C0'], extend='both')
    # surf.cmap.set_over('white')
    # surf.cmap.set_under('white')
    # surf.changed()



    # plt.clabel(surf, inline=False, fontsize=12, fmt ='%0.2f', colors = 'black')
    # ax.set_xlabel('$z_0$')
    # ax.set_ylabel('$z_1$')
    # ax.set_title(p)
    # cbar = plt.colorbar(surf)
    # cbar.set_label('({:s})'.format(str(p)))
    # plt.show()
    # plt.savefig('./figures/{:s}_latentFieldContours.pdf'.format(p), dpi=200, bbox_inches='tight')

  #-------------------------------------------#




In [184]:
scatter_contour = copy.deepcopy(scatter_plotly)


prop = ['ID']
contour_trace = plotlyLatentContours(prop)

scatter_contour.add_trace(contour_trace)
scatter_contour.update_layout(title=(f'{prop}'))
scatter_contour.show(renderer='colab')


In [ ]:
def plotlyLatentContours():
  n = 80
  zmin, zmax = -3,3
  xi = np.linspace(zmin, zmax, n)
  yi = np.linspace(zmin, zmax, n)
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim'])).to('cpu')
  Z[:,0], Z[:,1] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))

  vae = materialEncoder.vaeNet.to('cpu')
  trainData_z_np = to_np(vae.encoder.z)
  decoded = vae.decoder(Z)



  #-------------------------------------------#
  props = ['RPM']
  cutOff = [30000,40000];


  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matPropVal = to_np(10.**(scaleMin + decoded[:,idx]*(scaleMax - scaleMin)))
    levs = np.logspace(np.log10(min(matPropVal))*0.5, np.log10(max(matPropVal)), 40)
    contour_trace = go.Contour(z= matPropVal.reshape((n,n)), x=xi, y=yi,
        colorscale='Viridis',
        showscale=True,  # Hide the color scale
        contours=dict(coloring='lines', showlabels=True)
    )


    return contour_trace



In [ ]:
contour_trace = plotlyLatentContours()
scatter_contour = scatter_plotly.add_trace(contour_trace)
scatter_contour.show(renderer='colab')

In [ ]:
def plotLatentWithPropertyNew(ltnt1 = 0, ltnt2 = 1):
  n = 80
  zmin, zmax = -3,3
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim'])).to('cpu')
  Z[:,ltnt1], Z[:,ltnt2] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))

  vae = materialEncoder.vaeNet.to('cpu')
  trainData_z_np = to_np(vae.encoder.z)
  decoded = vae.decoder(Z)



  #-------------------------------------------#
  props = ['ID','RPM']
  cutOff = [[0.25,1.0],[30000,40000]];
  fig, ax = plotLatent(ltnt1 = ltnt1, ltnt2 = ltnt2, plotHull = False, plotEllipse=True, annotateHead = False, saveFileName = './figures/SpringLatent.pdf')
  cols = ['g','b']

  i = 0
  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matPropVal = to_np(10.**(scaleMin + decoded[:,idx]*(scaleMax - scaleMin)))


    levs = np.logspace(np.log10(min(matPropVal))*0.5, np.log10(max(matPropVal)), 20)
    surf = ax.contourf(X, Y, ((matPropVal).reshape((n,n))), levels = cutOff[i], alpha = 0.4, colors=[cols[i],cols[i]], extend='both')
    i+=1;

    # surf = ax.contour(X, Y, (matPropVal).reshape((n,n)), levels = cutOff, cmap='viridis_r', alpha = 0.6)
    surf.cmap.set_over('white')
    surf.cmap.set_under('white')
    surf.changed()

    plt.clabel(surf, inline=False, fontsize=10, fmt ='%0.0f', colors = 'black')
    ax.set_xlabel('$z_0$')
    ax.set_ylabel('$z_1$')
    ax.set_title('Bearing selector: ID:0.25-1 in & ROM: 30000-40000')
    cbar = plt.colorbar(surf, pad=0.01)
    plt.clabel(surf, inline=False, fontsize=10, fmt ='%0.2f', colors = 'black')
    cbar.set_label('({:s})'.format(str(p)), rotation=270, labelpad=0)
    plt.show()

    plt.savefig('./figures/{:s}Combined_latentField.pdf'.format(p), dpi=200, bbox_inches='tight')

  #-------------------------------------------#


plt.close('all')
plotLatentWithPropertyNew()